In [17]:
import json
import os
from dotenv import load_dotenv
from datetime import datetime
from pathlib import Path
from time import sleep

from pydantic import BaseModel, ValidationError

In [18]:
load_dotenv('../.env')

True

In [19]:
TASK_NAME = "relevance_check_v0"

def get_run_id():
    return os.getenv('RUNID') 

RUNID = get_run_id()

INPUT_DATA_PATH = f"../local_tests_data/azure_openai_batch_processing_files/{RUNID}/{TASK_NAME}/OUTPUTS/"

RUN_TIME = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')

OUTPUT_DATA_PATH = f"../local_tests_data/relevant_articles_list/{RUNID}/"

os.makedirs(OUTPUT_DATA_PATH, exist_ok=True)

print(f"Run ID: {RUNID} at {RUN_TIME}")

Run ID: RUNID_2 at 2025-06-02 15:43:52


In [20]:
def read_outputs():
    outputs = []
    for filename in os.listdir(INPUT_DATA_PATH):
        if filename.endswith('.jsonl'):
            with open(os.path.join(INPUT_DATA_PATH, filename), 'r') as f:
                    lines_raw = f.readlines()
                    for line in lines_raw:
                        output_dict = json.loads(line)
                        model = output_dict.get("response").get("body").get("model")
                        line_id = output_dict.get("custom_id")
                        run_id, task_name, article_id = line_id.split("--")
                        content_json = output_dict.get("response").get("body").get("choices")[0].get("message").get("content")
                        content = json.loads(content_json)
                        outputs.append({
                            "model": model,
                            "run_id": run_id,
                            "task_name": task_name,
                            "article_id": article_id,
                            "relevance": content.get("relevance"),
                            "article_language": content.get("article_language"),
                        })
    return outputs

def get_relevant_articles():

    outputs = read_outputs()

    with open(f'../local_tests_data/raw_articles_list/{RUNID}/raw_articles_list_{RUNID}.json', 'r') as f:
        raw_articles_list = json.load(f)

    raw_articles_dict = {}
    for a in raw_articles_list:
        a.pop("model")
        a.pop("task_name")
        article_id = a.get("article_id")
        raw_articles_dict[article_id] = a

    relevant_articles = []
    for output in outputs:
        if output.get("relevance") != 2:
            continue

        article_id = output.get("article_id")
        a = raw_articles_dict[article_id]

        relevant_article = output | a
        relevant_articles.append(relevant_article)

    return relevant_articles

In [21]:
relevant_articles = get_relevant_articles()

In [22]:
relevant_articles

[{'model': 'gpt-4o-2024-11-20',
  'run_id': 'RUNID_2',
  'task_name': 'relevance_check_v0',
  'article_id': 'techcrunch_20250530144755886209',
  'relevance': 2,
  'article_language': 'en',
  'source_name': 'techcrunch',
  'article_title': 'Hugging Face unveils two new humanoid robots',
  'article_url': 'https://techcrunch.com/2025/05/29/hugging-face-unveils-two-new-humanoid-robots/',
  'article_keywords': ['Robotics'],
  'crawled_at': '2025-05-30 14:47:55'},
 {'model': 'gpt-4o-2024-11-20',
  'run_id': 'RUNID_2',
  'task_name': 'relevance_check_v0',
  'article_id': 'itespresso_20250530144756051666',
  'relevance': 2,
  'article_language': 'es',
  'source_name': 'itespresso',
  'article_title': 'Apple lanzó discretamente un LLM multimodal de código abierto en octubre',
  'article_url': 'https://www.itespresso.es/apple-llm-multimodal-open-source-243999.html',
  'article_keywords': ['Apple', 'LLM código abierto'],
  'crawled_at': '2025-05-30 14:47:55'},
 {'model': 'gpt-4o-2024-11-20',
  'r

In [23]:
def save_outputs(relevant_articles):
    with open(os.path.join(OUTPUT_DATA_PATH, f"relevant_articles_list_{RUNID}.json"), 'w') as f:
        json.dump(relevant_articles, f, indent=4)
    print(f"Relevant articles saved to {OUTPUT_DATA_PATH}relevant_articles_list_{RUNID}.json")

In [24]:
save_outputs(relevant_articles)

Relevant articles saved to ../local_tests_data/relevant_articles_list/RUNID_2/relevant_articles_list_RUNID_2.json
